In [1]:
%load_ext autoreload
%autoreload 2
import requests
import json


from greenvisor.backend.utils.prompt_json import strip_answer, validate_json, prompt_en, form_renovations_json

## Query for recommendations

In [3]:
url = '<IP:port/graph/query>'
myobj = {
    #'question': 'Which PV should I install?',
    'question': prompt_en,
    'lang': 'english',
    'userid': '<USERID>'
}

x = requests.post(url, json = myobj)

In [4]:
text = """```json
"hello world"
```"""
print(strip_answer(text))


"hello world"



In [5]:
answer = strip_answer(json.loads(x.text)["message"])
print(answer)
if not validate_json(answer):
    print("malformed json")

[
  {
    "Measure": "Insulation of exterior walls and roof",
    "ExampleProduct": "Rockwool insulation panels",
    "InvestmentCostsEuro": 25000,
    "HeatingCostPerYear": {
      "current": 2200,
      "future": 1400
    },
    "CO2KgPerYear": {
      "current": 8000,
      "future": 4000
    },
    "SubsidiesPercentage": {
      "maximum": 30,
      "comment": "Up to 30% subsidy possible via BEG (Bundesförderung für effiziente Gebäude) with additional 5% iSFP bonus and 5% climate speed bonus"
    },
    "obligatory": {
      "value": true,
      "comment": "According to GEG §47, top-floor ceilings and accessible attic spaces must be insulated if they do not meet minimum thermal insulation standards."
    },
    "RecommendedChangeDate": {
      "value": 2024
    }
  },
  {
    "Measure": "Renovate facade",
    "ExampleProduct": "High-quality wood facade treatment",
    "InvestmentCostsEuro": 15000,
    "HeatingCostPerYear": {
      "current": 1400,
      "future": 1200
    },
    "C

In [9]:
rename_cols = {
    'Measure': 'Renovation',
    'ExampleProduct': 'Example product',
    'InvestmentCostsEuro': 'Investment, €',
    'HeatingCostPerYear': 'Heating costs, kWh/year',
    'CO2KgPerYear': 'CO2 emissions, kg/year',
    'SubsidiesPercentage': 'Subsidies',
    'obligatory': 'Obligatory by law',
    'RecommendedChangeDate': 'Planned date',
}

renovations_raw = json.loads(answer)
renovations_table = form_renovations_json(renovations_raw)

print_table = {}
for old_col, new_col in rename_cols.items():
    print_table[new_col] = renovations_table[old_col]

print_table

{'Renovation': ['Insulation of exterior walls and roof',
  'Renovate facade',
  'Changing heating type to heat pump',
  'Install solar thermal system'],
 'Example product': ['Rockwool insulation panels',
  'High-quality wood facade treatment',
  'Vaillant aroTHERM plus',
  'Viessmann Vitosol 200-FM'],
 'Investment, €': ['25000', '15000', '18000', '9000'],
 'Heating costs, kWh/year': ['from 2200 €/year to 1400 €/year',
  'from 1400 €/year to 1200 €/year',
  'from 1200 €/year to 650 €/year',
  'from 650 €/year to 450 €/year'],
 'CO2 emissions, kg/year': ['from 8000 kg/year to 4000 kg/year',
  'from 4000 kg/year to 3400 kg/year',
  'from 3400 kg/year to 900 kg/year',
  'from 900 kg/year to 600 kg/year'],
 'Subsidies': ['Up to 30% subsidy possible via BEG (Bundesförderun...',
  'Up to 25% subsidy via BEG including bonuses for fa...',
  'Up to 70% subsidy possible for heat pumps via BEG ...',
  'Up to 25% subsidy via BEG for solar thermal system...'],
 'Obligatory by law': ['yes', 'no', 'ye

In [11]:
from greenvisor.backend.utils.prompt_json import strip_answer, validate_json, prompt_en, form_renovations_json
from greenvisor.backend.pdf_export.pdfexport import json_to_pdf
json_to_pdf("Renovation proposals", print_table, "renovations.pdf")

## Query for totals

In [12]:
plan = "\n--- BEGIN OF HOUSE RENOVATION PLAN ---\n" + answer + "\n--- END OF HOUSE RENOVATION PLAN ---\n"

In [13]:
from greenvisor.backend.utils.prompt_json import prompt_total_en
myobj = {
    'question': prompt_total_en + plan,
    'lang': 'en',
    'userid': '681fc0844c6fdb1b20cd'
}

total_ret = requests.post(url, json = myobj)

In [14]:
total = strip_answer(json.loads(total_ret.text)["message"])
print(total)
if not validate_json(total):
    print("malformed json")

[
  {
    "TotalHeatingUsage": {
        "before": 20000,
        "after": 7000
    },
    "TotalElectricityUsage": {
        "before": 3500,
        "after": 5500
    },
    "TotalHeatingCost": {
        "before": 2200,
        "after": 450
    },
    "TotalElectricityCost": {
        "before": 1200,
        "after": 1550
    },
    "EnergyEfficiency": {
        "before": "G",
        "after": "B"
    },
    "CO2Emissions": {
        "before": 8000,
        "after": 600
    }
  }
]


## Present totals

In [21]:
from greenvisor.backend.utils.prompt_json import form_total_json

def present_total(renovations_raw, total_raw):
    renovations_raw = json.loads(answer)
    renovations_table = form_renovations_json(renovations_raw)

    # calculate total investment
    total_investment = 0
    investment_with_subsidy = 0
    for measure in renovations_raw:
        total_investment += measure["InvestmentCostsEuro"]
        investment_with_subsidy += measure["InvestmentCostsEuro"] * measure['SubsidiesPercentage']['maximum'] / 100
    #print(total_investment, investment_with_subsidy)

    # calculate savings
    total_savings = 0
    for measure in renovations_raw:
        total_savings += measure['HeatingCostPerYear']['current'] - measure['HeatingCostPerYear']['future']
    #print(total_savings)

    
    # amortisation (payoff)
    amortisation_years = investment_with_subsidy / total_savings
    average_subsidy = investment_with_subsidy / total_investment
    #print(amortisation_years)
    #print(average_subsidy * 100)

    # fill in pdf
    total_show = form_total_json(json.loads(total_raw))
    appendix = [
        ['TotalInvestment', '-', f"{total_investment} €"],
        ['Subsidy',         '-', str(int(average_subsidy * 100)) + "%"],
        ['Savings',         '-', str(int(total_savings))],
        ['Amortisation',    '-', str(int(amortisation_years))]
    ]
    
    for i in range(len(appendix)):
        for j, col in enumerate(total_show.keys()):
            total_show[col].append(appendix[i][j])

    aspects_rename = {
        'TotalHeatingUsage': 'Total heat usage, kWh/year',
        'TotalElectricityUsage': 'Total electricity usage, kWh/year',
        'TotalHeatingCost': 'Total heating cost, €/year',
        'TotalElectricityCost': 'Total electricity cost, €/year',
        'EnergyEfficiency': 'Energy efficiency',
        'CO2Emissions': 'CO2 emissions, kg/year',
        'TotalInvestment': 'Total investment, €',
        'Subsidy': 'Subsidy',
        'Savings': 'Savings per month, €',
        'Amortisation': 'Amortisation, years'
    }

    total_show['aspect'] = list(aspects_rename.values())
        
    
    return total_show

In [22]:
present_total(renovations_raw, total)

{'aspect': ['Total heat usage, kWh/year',
  'Total electricity usage, kWh/year',
  'Total heating cost, €/year',
  'Total electricity cost, €/year',
  'Energy efficiency',
  'CO2 emissions, kg/year',
  'Total investment, €',
  'Subsidy',
  'Savings per month, €',
  'Amortisation, years'],
 'before': ['20000', '3500', '2200', '1200', 'G', '8000', '-', '-', '-', '-'],
 'after': ['7000',
  '5500',
  '450',
  '1550',
  'B',
  '600',
  '67000 €',
  '38%',
  '1750',
  '14']}

In [23]:
from greenvisor.backend.pdf_export.pdfexport import json_to_pdf
json_to_pdf('Saving summary', present_total(renovations_raw, total), "summary.pdf")